In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [5]:
def fen_to_feature_array(fen: str):
    game_attributes = fen.split()

    assert len(game_attributes) == 6

    board, turn, castling, en_passant, halfmove, fullmove = game_attributes

    piece_map = {
        'p': 1, 'n': 2, 'b': 3, 'r': 4, 'q': 5, 'k': 6,
        'P': 7, 'N': 8, 'B': 9, 'R': 10, 'Q': 11, 'K': 12
    }

    board_vector = []
    for row in board.split('/'):
        row_data = []
        for ch in row:
            if ch.isdigit():
                row_data.extend([0] * int(ch))
            else:
                row_data.append(piece_map[ch])
        board_vector.extend(row_data)

    turn_val = 0 if turn == 'w' else 1

    castling_vec = [int(c in castling) for c in 'KQkq']

    if en_passant != '-':
        file = ord(en_passant[0]) - ord('a')
        rank = int(en_passant[1]) - 1
        en_passant_val = rank * 8 + file
    else:
        en_passant_val = -1

    halfmove_val = int(halfmove)
    fullmove_val = int(fullmove)

    return board_vector + [turn_val] + castling_vec + [en_passant_val, halfmove_val, fullmove_val]

In [6]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [7]:
class ChessDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [8]:
class ChessEvalNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [9]:
df = pd.read_csv("../data/chess_data.csv")

X = df['FEN'].apply(fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [10]:
model = ChessEvalNN(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 927219.9411
Epoch 2, Loss: 850644.8584
Epoch 3, Loss: 735805.7372
Epoch 4, Loss: 614551.8796
Epoch 5, Loss: 512876.8745
Epoch 6, Loss: 448385.6092
Epoch 7, Loss: 394669.8345
Epoch 8, Loss: 361629.4191
Epoch 9, Loss: 328408.9472
Epoch 10, Loss: 298896.5376
Epoch 11, Loss: 269858.6813
Epoch 12, Loss: 257559.2605
Epoch 13, Loss: 240418.3916
Epoch 14, Loss: 222882.6354
Epoch 15, Loss: 217015.7272
Epoch 16, Loss: 202830.9199
Epoch 17, Loss: 196762.8443
Epoch 18, Loss: 182347.3733
Epoch 19, Loss: 177794.7490
Epoch 20, Loss: 166972.8774


In [11]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 675.1118


In [12]:
torch.save(model.state_dict(), 'neural_network.pth')